In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.stats import iqr
from scipy.stats import entropy
from scipy.stats import mode
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv("sum_df.csv")
df

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle
...,...,...,...,...
193855,5.109226,-15.452178,-1.470040,walking
193856,6.373365,-11.741165,-8.226476,walking
193857,3.289633,-9.993398,-0.383072,walking
193858,-2.978387,-3.050213,1.273715,walking


In [3]:
def time_features(p):
    features = []

    features.append(np.mean(p))
    features.append(np.median(p))
    features.append(np.max(p))
    features.append(np.min(p))
    features.append(np.sum(np.abs(p)))

    features.append(np.sum(p ** 2))  # P
    features.append(np.sum(np.abs(p) ** 2))  # Е

    return features

In [4]:
Xx = df['accelerometer_X']
Xy = df['accelerometer_Y']
Xz = df['accelerometer_Z']

In [15]:
Xx_df = Xx.apply(lambda x: pd.Series(time_features(x)))
Xx_df.columns = ['x_mean', 'x_median', 'x_max', 'x_min', 'x_sum_abs', 'x_sum_squared', 'x_sum_abs_squared']
Xx_df

,x_mean,x_median,x_max,x_min,x_sum_abs,x_sum_squared,x_sum_abs_squared
0,1.000776,1.000776,1.000776,1.000776,1.000776,1.001553,1.001553
1,0.718261,0.718261,0.718261,0.718261,0.718261,0.515899,0.515899
2,-0.909797,-0.909797,-0.909797,-0.909797,0.909797,0.827731,0.827731
3,5.099650,5.099650,5.099650,5.099650,5.099650,26.006430,26.006430
4,1.762132,1.762132,1.762132,1.762132,1.762132,3.105109,3.105109
...,...,...,...,...,...,...,...
193855,5.109226,5.109226,5.109226,5.109226,5.109226,26.104190,26.104190
193856,6.373365,6.373365,6.373365,6.373365,6.373365,40.619781,40.619781
193857,3.289633,3.289633,3.289633,3.289633,3.289633,10.821685,10.821685
193858,-2.978387,-2.978387,-2.978387,-2.978387,2.978387,8.870789,8.870789


In [16]:
Xy_df = Xy.apply(lambda y: pd.Series(time_features(y)))
Xy_df.columns = ['y_mean', 'y_median', 'y_max', 'y_min', 'y_sum_abs', 'y_sum_squared', 'y_sum_abs_squared']
Xy_df

,y_mean,y_median,y_max,y_min,y_sum_abs,y_sum_squared,y_sum_abs_squared
0,4.616021,4.616021,4.616021,4.616021,4.616021,21.307650,21.307650
1,4.209007,4.209007,4.209007,4.209007,4.209007,17.715740,17.715740
2,-0.282516,-0.282516,-0.282516,-0.282516,0.282516,0.079815,0.079815
3,0.148441,0.148441,0.148441,0.148441,0.148441,0.022035,0.022035
4,-0.162806,-0.162806,-0.162806,-0.162806,0.162806,0.026506,0.026506
...,...,...,...,...,...,...,...
193855,-15.452178,-15.452178,-15.452178,-15.452178,15.452178,238.769805,238.769805
193856,-11.741165,-11.741165,-11.741165,-11.741165,11.741165,137.854956,137.854956
193857,-9.993398,-9.993398,-9.993398,-9.993398,9.993398,99.868004,99.868004
193858,-3.050213,-3.050213,-3.050213,-3.050213,3.050213,9.303799,9.303799


In [17]:
Xz_df = Xz.apply(lambda z: pd.Series(time_features(z)))
Xz_df.columns = ['z_mean', 'z_median', 'z_max', 'z_min', 'z_sum_abs', 'z_sum_squared', 'z_sum_abs_squared']
Xz_df

,z_mean,z_median,z_max,z_min,z_sum_abs,z_sum_squared,z_sum_abs_squared
0,8.576031,8.576031,8.576031,8.576031,8.576031,73.548308,73.548308
1,8.446744,8.446744,8.446744,8.446744,8.446744,71.347484,71.347484
2,9.203311,9.203311,9.203311,9.203311,9.203311,84.700933,84.700933
3,8.418014,8.418014,8.418014,8.418014,8.418014,70.862960,70.862960
4,9.251195,9.251195,9.251195,9.251195,9.251195,85.584609,85.584609
...,...,...,...,...,...,...,...
193855,-1.470040,-1.470040,-1.470040,-1.470040,1.470040,2.161018,2.161018
193856,-8.226476,-8.226476,-8.226476,-8.226476,8.226476,67.674907,67.674907
193857,-0.383072,-0.383072,-0.383072,-0.383072,0.383072,0.146744,0.146744
193858,1.273715,1.273715,1.273715,1.273715,1.273715,1.622350,1.622350


In [19]:
Xdf = df[['accelerometer_X','accelerometer_Y','accelerometer_Z']]
X = pd.concat([Xdf, Xx_df, Xy_df, Xz_df], axis=1)
X

,accelerometer_X,accelerometer_Y,accelerometer_Z,x_mean,x_median,x_max,x_min,x_sum_abs,x_sum_squared,x_sum_abs_squared,...,y_sum_abs,y_sum_squared,y_sum_abs_squared,z_mean,z_median,z_max,z_min,z_sum_abs,z_sum_squared,z_sum_abs_squared
0,1.000776,4.616021,8.576031,1.000776,1.000776,1.000776,1.000776,1.000776,1.001553,1.001553,...,4.616021,21.307650,21.307650,8.576031,8.576031,8.576031,8.576031,8.576031,73.548308,73.548308
1,0.718261,4.209007,8.446744,0.718261,0.718261,0.718261,0.718261,0.718261,0.515899,0.515899,...,4.209007,17.715740,17.715740,8.446744,8.446744,8.446744,8.446744,8.446744,71.347484,71.347484
2,-0.909797,-0.282516,9.203311,-0.909797,-0.909797,-0.909797,-0.909797,0.909797,0.827731,0.827731,...,0.282516,0.079815,0.079815,9.203311,9.203311,9.203311,9.203311,9.203311,84.700933,84.700933
3,5.099650,0.148441,8.418014,5.099650,5.099650,5.099650,5.099650,5.099650,26.006430,26.006430,...,0.148441,0.022035,0.022035,8.418014,8.418014,8.418014,8.418014,8.418014,70.862960,70.862960
4,1.762132,-0.162806,9.251195,1.762132,1.762132,1.762132,1.762132,1.762132,3.105109,3.105109,...,0.162806,0.026506,0.026506,9.251195,9.251195,9.251195,9.251195,9.251195,85.584609,85.584609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193855,5.109226,-15.452178,-1.470040,5.109226,5.109226,5.109226,5.109226,5.109226,26.104190,26.104190,...,15.452178,238.769805,238.769805,-1.470040,-1.470040,-1.470040,-1.470040,1.470040,2.161018,2.161018
193856,6.373365,-11.741165,-8.226476,6.373365,6.373365,6.373365,6.373365,6.373365,40.619781,40.619781,...,11.741165,137.854956,137.854956,-8.226476,-8.226476,-8.226476,-8.226476,8.226476,67.674907,67.674907
193857,3.289633,-9.993398,-0.383072,3.289633,3.289633,3.289633,3.289633,3.289633,10.821685,10.821685,...,9.993398,99.868004,99.868004,-0.383072,-0.383072,-0.383072,-0.383072,0.383072,0.146744,0.146744
193858,-2.978387,-3.050213,1.273715,-2.978387,-2.978387,-2.978387,-2.978387,2.978387,8.870789,8.870789,...,3.050213,9.303799,9.303799,1.273715,1.273715,1.273715,1.273715,1.273715,1.622350,1.622350


In [20]:
activity = df['activity']
activity

0            idle
1            idle
2            idle
3            idle
4            idle
           ...   
193855    walking
193856    walking
193857    walking
193858    walking
193859    walking
Name: activity, Length: 193860, dtype: object

In [21]:
y = []
for i in activity:
  if i == 'idle':
    y.append(1)
  if i == 'running':
    y.append(2)
  if i == 'stairs':
    y.append(3)
  if i == 'walking':
    y.append(4)
print(y)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

SVC()

In [26]:
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8872


In [27]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [30]:
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9997
